### Data Dictionary

**Beetles table**
* ID: beetle ID (unique integer)
* Group: flight test group (A, B, C, D)
* Sex: M, F
* date_treated
* date_dead
* OrNV detected: t, f
* Notes

**Observations table**

This table is not in standard format. It is a matrix with 'ID' as the first column.
The remaining columns labeled '2023/02/28', '2023/03/01', ... contain observation codes for each beetle.
The dates indicate when the observation was made. 
For example, an F in column 2023/02/28 indicates that this beetle flew during the previous night ( 2023-02-27 19:00 to 2023-02-28 07:00. 

* F: beetle flew (collected from bottom of chamber)
* N: beetle did not fly (collected from paint bucket)
* D: dead
* M: missing

In [31]:
import pandas as pd
import plotly.express as px

# GET DATA

In [ ]:
SHEET_ID = '1jwgm7h_-Al4MspsfC4sP6E03QrjpZcTPr2JC7WLU2QM'

In [ ]:
def get_google_sheet(sheet_id, sheet_name):
    """
    Returns a data frame generated from a Google sheet
    """
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    return pd.read_csv(url)

# SHEET_ID = '1jwgm7h_-Al4MspsfC4sP6E03QrjpZcTPr2JC7WLU2QM'
# get_google_sheet(SHEET_ID, 'Beetles')

In [ ]:
# get mass table

# try:

df_mass = get_google_sheet(SHEET_ID, 'Mass')
df_mass = pd.melt(df_mass, id_vars=['Unnamed: 0'], var_name='date', value_name='milligrams')
df_mass.rename({'Unnamed: 0':'ID'}, axis='columns', inplace=True)
df_mass.dropna(inplace=True)    
# save to disk
df_mass.to_csv('mass.csv', index=False)
print('Mass sheet downloaded and saved to mass.csv')

# except Exception as e: 
#     print(e)
#     print('WARNING: Failed to get beetles table from Google sheets; Reading csv file from disk')
#     df_beetles = pd.read_csv('mass.csv')
df_mass

In [ ]:
# get 'beetles' table

# try:

df_beetles = get_google_sheet(SHEET_ID, 'Beetles')    
# remove columns after 'Notes'
df_beetles = df_beetles.loc[:,:'Notes']
# Convert dates from string to datetime
df_beetles['date treated'] = pd.to_datetime(df_beetles['date treated'], format='%Y/%m/%d')
df_beetles['date dead'] = pd.to_datetime(df_beetles['date dead'], format='%Y/%m/%d')
# save to disk
df_beetles.to_csv('beetles.csv', index=False)
print('Beetles sheet downloaded and saved to beetles.csv')
    
# except Exception as e: 
#     print(e)
#     print('WARNING: Failed to get beetles table from Google sheets; Reading csv file from disk')
#     df_beetles = pd.read_csv('beetles.csv')
df_beetles

In [ ]:
# get observations table

# try:

df_observations = get_google_sheet(SHEET_ID, 'Observations')
# No idea why the first column is unnamed, but this fixes the problem
df_observations.rename(columns={'Unnamed: 0':'ID'}, inplace=True)
# Drop all columns which do not contain data
df_observations.dropna(axis='columns', how='all', inplace=True)
# Convert the table into a more standard format
df_observations = df_observations.melt(id_vars='ID')
# Rename
df_observations.rename(columns={'variable':'date', 'value':'obs'}, inplace=True)
# Convert date from string to datetime
df_observations['date'] = pd.to_datetime(df_observations['date'], format='%Y/%m/%d')    
df_observations.to_csv('observations.csv', index=False)
print('Observations sheet downloaded and saved to observations.csv')
    
# except Exception as e: 
#     print(e)
#     print('WARNING: Failed to get observations sheet from Google sheets; Reading csv file from disk')
#     df_observations = pd.read_csv('observations.csv')
df_observations

In [ ]:
df_observations.obs.unique()

In [ ]:
# Merge tables using ID as key

df_merged = df_beetles.merge(df_observations)

# Drop all columns which do not contain data
df_merged.dropna(axis='columns', how='all', inplace=True)

# Select the rows we are interested in
# df_merged = df_merged.query('obs.str.contains("F") | obs.str.contains("N")')

df_merged

In [ ]:
# Select rows where obs contains 'F' (flew) or 'N' (did not fly)
df_merged = df_merged.query("obs.str.contains('F') | obs.str.contains('N')")

# Create a new column 'flew'
df_merged['flew'] = df_merged.obs.str.contains('F')

df_merged

In [ ]:
# Create a new dataframe containing only the columns we need

df = df_merged[['Group','date','flew']]
df

In [ ]:
def proportion_flew(df):
    """ returns proportion of beetles which flew """
    return df.flew.sum() / df.flew.count()

# proportion_flew(df)

In [ ]:
# calculate proportion of beetles which flew for each 'Group' and 'date'
gb = df.groupby(['Group','date']).apply(proportion_flew)
gb

# PLOT

In [ ]:
df = gb.reset_index().rename(columns={0:'proportion_flew'})
fig = px.bar(df, x='date', y='proportion_flew', facet_row='Group', color='Group', color_discrete_sequence=['green','green','red','red'])

fig.update_layout(showlegend=False, width=1000, height=1000)

fig.add_vline(x=pd.to_datetime('2023-03-05 08').timestamp()*1000, line_dash="dot", row=1, col=1, annotation_text="food<br>+virus", annotation_position="top right")
fig.add_vline(x=pd.to_datetime('2023-03-05 08').timestamp()*1000, line_dash="dot", row=2, col=1, annotation_text="food<br>+virus", annotation_position="top right")
fig.add_vline(x=pd.to_datetime('2023-03-04 08').timestamp()*1000, line_dash="dot", row=3, col=1, annotation_text="food", annotation_position="top right")
fig.add_vline(x=pd.to_datetime('2023-03-03 08').timestamp()*1000, line_dash="dot", row=4, col=1, annotation_text="food", annotation_position="top right")

fig.add_vline(x=pd.to_datetime('2023-03-13 08').timestamp()*1000, line_dash="dot", row="all", col=1, annotation_text="food", annotation_position="top right")
fig.add_vline(x=pd.to_datetime('2023-03-21 08').timestamp()*1000, line_dash="dot", row="all", col=1, annotation_text="food", annotation_position="top right")
fig.add_vline(x=pd.to_datetime('2023-04-04 08').timestamp()*1000, line_dash="dot", row="all", col=1, annotation_text="food", annotation_position="top right")
   
fig.write_html("results-plot.html")
fig.write_image("results-plot.png")

fig.show()